In [1]:
%store -r __impC

In [2]:
__impC

In [3]:
%store -r __SNKF

In [4]:
__SNKF

READING DATA

In [5]:
train = pd.read_csv('Data/train.csv')
test = pd.read_csv('Data/test.csv')

In [6]:
def group_df(x,y):
    return pd.concat([x,y],sort=True).reset_index(drop=True) 

def split_df(x):
    return x[:891] , x[891:].drop('Survived',axis=1)   

In [7]:
df_all = group_df(train,test)
df_train = train.copy()
df_test = test.copy()

dfs = [df_train,df_test]

df_all.name = 'ALL DATA'
df_train.name = 'TRAIN DATA'
df_test.name = 'TEST DATA'

MISSING VALUES

In [8]:
df_all['Age'] = df_all.groupby(['Pclass','Sex'])['Age'].apply(lambda x : x.fillna(x.median()))

In [9]:
df_all['Embarked'] = df_all['Embarked'].fillna('S')    #Embarked

In [10]:
med_fare = df_all.groupby(['Pclass', 'Parch', 'SibSp']).Fare.median()[3][0][0]  #median fare
df_all['Fare'] = df_all['Fare'].fillna(med_fare)

In [11]:
df_all['Deck'] = df_all['Cabin'].apply(lambda x : x[0] if pd.isna(x) == False else x)
df_all['Deck'] = df_all['Deck'].replace('T','A')
df_all['Deck'] = df_all['Deck'].replace(['A','B','C'],'ABC')
df_all['Deck'] = df_all['Deck'].replace(['D','E'],'DE')
df_all['Deck'] = df_all['Deck'].replace(['F','G'],'FG')
df_all['Deck'] = df_all['Deck'].fillna('M')

In [12]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Age          1309 non-null   float64
 1   Cabin        295 non-null    object 
 2   Embarked     1309 non-null   object 
 3   Fare         1309 non-null   float64
 4   Name         1309 non-null   object 
 5   Parch        1309 non-null   int64  
 6   PassengerId  1309 non-null   int64  
 7   Pclass       1309 non-null   int64  
 8   Sex          1309 non-null   object 
 9   SibSp        1309 non-null   int64  
 10  Survived     891 non-null    float64
 11  Ticket       1309 non-null   object 
 12  Deck         1309 non-null   object 
dtypes: float64(3), int64(4), object(6)
memory usage: 133.1+ KB


Feature Engineering

In [13]:
df_all['Fare'] = pd.qcut(df_all['Fare'], 13)

In [14]:
df_all['Age'] = pd.qcut(df_all['Age'],10)

In [15]:
df_all['Family_Size'] = df_all[['SibSp','Parch']].apply(lambda x : 'Alone' if x[0]+x[1]+1 == 1 else 'Small' if 1< x[0]+x[1]+1 < 5 else 'Medium' if 4 < x[0]+x[1]+1 < 7 else 'Large',axis=1)  

In [16]:
df_all['Ticket_Frq'] = df_all.groupby('Ticket')['Ticket'].transform('count')

In [17]:
df_train,df_test = split_df(df_all) 

In [18]:
L = [1,2,3,4,5,6,7,8]
D = {1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,11:0}
for i in L:
    D[i] = D[i] + round(df_train.groupby(['Ticket_Frq','Survived'])['Age'].count()[i][1]/(df_train.groupby(['Ticket_Frq','Survived'])['Age'].count()[i][1]+df_train.groupby(['Ticket_Frq','Survived'])['Age'].count()[i][0]),2) 
    
D[11] = 0    

In [19]:
df_all['Ticket_sr'] = df_all['Ticket_Frq'].apply(lambda x : D[x] if x in D.keys() else x)

In [20]:
#df_all['Family'] = df_all['Name'].apply(lambda x : x.split(',')[0])

In [21]:
#df_train,df_test = split_df(df_all) 

In [22]:
#df_train.groupby(['Family','Survived']).count()['Age']

In [23]:
def get_title(name):
    return name.split(',')[1].split('.')[0]

In [24]:
df_all['Title'] = df_all['Name'].apply(get_title)

In [25]:
df_all['Title'].unique()

array([' Mr', ' Mrs', ' Miss', ' Master', ' Don', ' Rev', ' Dr', ' Mme',
       ' Ms', ' Major', ' Lady', ' Sir', ' Mlle', ' Col', ' Capt',
       ' the Countess', ' Jonkheer', ' Dona'], dtype=object)

In [26]:
df_all['Title'] = df_all['Title'].replace([' Miss', ' Ms', ' Mrs',' Lady',' the Countess',' Dona',' Mme'],'Female')
df_all['Title'] = df_all['Title'].replace([' Don',' Mlle',' Dr', ' Rev',' Major',' Sir',' Col',' Capt',' Jonkheer'],'Rare')

In [27]:
df_all['Married_Woman'] = df_all['Name'].apply(get_title).apply(lambda x : 1 if x == ' Mrs' else 0)

Feature Transforation

In [28]:
df_all = df_all.drop(['Cabin','Name','SibSp','PassengerId','Parch','Ticket','Ticket_Frq'],axis=1)

In [29]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Age            1309 non-null   category
 1   Embarked       1309 non-null   object  
 2   Fare           1309 non-null   category
 3   Pclass         1309 non-null   int64   
 4   Sex            1309 non-null   object  
 5   Survived       891 non-null    float64 
 6   Deck           1309 non-null   object  
 7   Family_Size    1309 non-null   object  
 8   Ticket_sr      1309 non-null   float64 
 9   Title          1309 non-null   object  
 10  Married_Woman  1309 non-null   int64   
dtypes: category(2), float64(2), int64(2), object(5)
memory usage: 96.0+ KB


In [30]:
def fact(X):
    for c in X.select_dtypes(object).columns:
        X[c] = X[c].factorize()[0]

In [31]:
fact(df_all)

In [32]:
#df_all.groupby('Family')['Family'].transform('count').unique()

In [33]:
cat_features = ['Embarked','Pclass','Deck','Family_Size','Title']

for feature in cat_features:
    df_feature = pd.get_dummies(df_all[feature],drop_first=True).to_numpy()
    col = []
    for i  in range(df_feature.shape[1]):
        col.append(f'{feature}_{i}')
    df_feature = pd.DataFrame(df_feature,columns=col)  
    df_all = pd.concat([df_all,df_feature],axis=1)
    df_all = df_all.drop(feature,axis=1)
    

In [34]:
df_all['Age'] = pd.DataFrame(pd.factorize(df_all['Age'])[0],columns=['Age'])

df_all['Fare'] = pd.DataFrame(pd.factorize(df_all['Fare'])[0],columns=['Fare'])

In [35]:
scaler = StandardScaler()
df_all['Age'] = pd.DataFrame(scaler.fit_transform(df_all['Age'].values.reshape(-1,1)),columns=['Age'])
df_all['Fare'] = pd.DataFrame(scaler.fit_transform(df_all['Fare'].values.reshape(-1,1)),columns=['Fare'])

In [36]:
df_all

,Age,Fare,Sex,Survived,Ticket_sr,Married_Woman,Embarked_0,Embarked_1,Pclass_0,Pclass_1,Deck_0,Deck_1,Deck_2,Family_Size_0,Family_Size_1,Family_Size_2,Title_0,Title_1,Title_2
0,-1.683026,-1.594202,0,0.0,0.27,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,-1.323457,-1.325249,1,1.0,0.51,1,1,0,0,0,1,0,0,0,0,0,1,0,0
2,-0.963887,-1.056295,1,1.0,0.27,0,0,0,0,1,0,0,0,1,0,0,1,0,0
3,-1.323457,-0.787341,1,1.0,0.51,1,0,0,0,0,1,0,0,0,0,0,1,0,0
4,-1.323457,-1.056295,0,0.0,0.27,0,0,0,0,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,-0.604318,-1.056295,0,NaN,0.27,0,0,0,0,1,0,0,0,1,0,0,0,0,0
1305,-1.323457,1.095333,1,NaN,0.65,0,1,0,0,0,1,0,0,1,0,0,1,0,0
1306,-1.323457,-1.594202,0,NaN,0.27,0,0,0,0,1,0,0,0,1,0,0,0,0,0
1307,-0.604318,-1.056295,0,NaN,0.27,0,0,0,0,1,0,0,0,1,0,0,0,0,0


In [37]:
df_train , df_test = split_df(df_all)

In [38]:
df_train_X = df_train.drop('Survived',axis=1)
df_train_y = train['Survived']

In [39]:
Models = [
            {0:LogisticRegression(),1:RandomForestClassifier(n_estimators=100,max_depth=1),2:XGBClassifier(max_depth=1),3:CatBoostClassifier(verbose=0),
             4:LinearSVC(),5:DecisionTreeClassifier(),6:RandomForestClassifier(n_estimators=100,max_depth=3),7:XGBClassifier(max_depth=4)},
    
            {0:LogisticRegression(),1:LinearSVC(),2:DecisionTreeClassifier(),3:RandomForestClassifier(n_estimators=100,max_depth=1),
             4:RandomForestClassifier(n_estimators=100,max_depth=4)},
    
            {0:LogisticRegression(),1:LinearSVC(),2:RandomForestClassifier(n_estimators=100,max_depth=1),
             3:RandomForestClassifier(n_estimators=100,max_depth=4)},
    
            {0:LogisticRegression()}
         
         ]

In [40]:
snet = SNKF(Models,9)

In [41]:
snet.fit(df_train_X,df_train_y)


 Accuracy Score of Layer-0 Model-0 = 0.8271604938271605
--------------------------------------------------


 Accuracy Score of Layer-0 Model-1 = 0.7721661054994389
--------------------------------------------------


 Accuracy Score of Layer-0 Model-2 = 0.8237934904601572
--------------------------------------------------


 Accuracy Score of Layer-0 Model-3 = 0.8271604938271605
--------------------------------------------------


 Accuracy Score of Layer-0 Model-4 = 0.8282828282828283
--------------------------------------------------


 Accuracy Score of Layer-0 Model-5 = 0.8047138047138047
--------------------------------------------------


 Accuracy Score of Layer-0 Model-6 = 0.8092031425364759
--------------------------------------------------


 Accuracy Score of Layer-0 Model-7 = 0.8305274971941639
--------------------------------------------------


 Layer 1 Starts Here: --1--1--1--1--1--1--1--1--1--- 


 Accuracy Score of Layer-1 Model-0 = 0.8260381593714927
---------------

In [42]:
#pickle.dump(snet,open('model-6-6-2-1_sub-2','wb'))
#c= pickle.load(open('model-6-6-2-1_sub-2','rb'))

In [43]:
final_predictions = snet.predict(df_test)

In [44]:
# MODEL_NAME = f'model-6-6-2-1{int(time.time())}'
# with open('model.log','a') as f:
#     f.write(f'{MODEL_NAME}\n')
#     for i in range(len(final_predictions[0])):    
#         f.write(f'{final_predictions[0][i]}\n')

In [45]:
final_predictions.columns = ['Survived']

In [46]:
# with open('model.log') as f:
#     content = f.read().splitlines()
# s1 = content[1:419]
# s2 = content[420:]
# x=0
# for i in range(418):
#     if s1[i] == s2[i]:
#        x+=1 
# print(x)    

In [47]:
final_predictions = pd.concat([test['PassengerId'],final_predictions],axis=1)

In [48]:
final_predictions.to_csv('Data/submission.csv',index=False)